In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
max_fatures = 1000


def load_data():
    print("Loading Data...")
    TwitterEmotion = pd.read_csv('text_emotion.csv')
    print("Data Loaded")
    return (TwitterEmotion['content'], array(TwitterEmotion['sentiment']))


#Below the data is sorted into nine emotion groups. Eight of the groups are the outter layer of the wheel, or the combinations of 
#two emotion groups. The ninth group is "Ambiguous" and "Neutral" put together.
def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'enthusiam' or emo == 'love' or emo == 'fun' or emo == 'happiness' or emo == 'relief' or emo == 'surprise'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'sadness' or emo == 'empty' or emo == 'worry' or emo == 'hate' or emo == 'boredom'):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.2

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data
### top_predictions variable changes how many predictions given
def test(model, X_test, tokenized_data, sentence_list, emotion_list, top_predictions=1):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            print("\n\n")

            
            pos = list(predictions[i]).index(max(predictions[i]))
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
            temp = predictions[i]
            
            print(sentence_list[i])

            for j in range(top_predictions):
                pos = list(temp).index(max(temp))
                print("\n # %s Predicted emotion : ",j+1, unique(emotion_list)[pos])
                temp[pos] = 0
            print("Actual emotion   : ", emotion_list[i])

    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/10


 6464/24394 [======>.......................] - ETA: 1:00:30 - loss: 0.6920 - acc: 0.53 - ETA: 31:51 - loss: 0.6922 - acc: 0.5469 - ETA: 22:18 - loss: 0.6931 - acc: 0.48 - ETA: 17:28 - loss: 0.6917 - acc: 0.53 - ETA: 14:37 - loss: 0.6925 - acc: 0.51 - ETA: 12:55 - loss: 0.6936 - acc: 0.51 - ETA: 11:30 - loss: 0.6924 - acc: 0.51 - ETA: 10:26 - loss: 0.6935 - acc: 0.49 - ETA: 9:36 - loss: 0.6926 - acc: 0.5000 - ETA: 8:58 - loss: 0.6920 - acc: 0.512 - ETA: 8:26 - loss: 0.6913 - acc: 0.517 - ETA: 7:58 - loss: 0.6928 - acc: 0.505 - ETA: 7:35 - loss: 0.6929 - acc: 0.507 - ETA: 7:22 - loss: 0.6922 - acc: 0.511 - ETA: 7:10 - loss: 0.6921 - acc: 0.510 - ETA: 6:55 - loss: 0.6919 - acc: 0.513 - ETA: 6:51 - loss: 0.6916 - acc: 0.518 - ETA: 6:38 - loss: 0.6915 - acc: 0.519 - ETA: 6:27 - loss: 0.6918 - acc: 0.518 - ETA: 6:16 - loss: 0.6919 - acc: 0.517 - ETA: 6:06 - loss: 0.6917 - acc: 0.519 - ETA: 5:57 - loss: 0.6910 - acc: 0.524 - ETA: 5:48 - loss: 0.6913 - acc: 0.520 - ETA: 5:40 - loss: 0.6908 - a

12992/24394 [==============>...............] - ETA: 2:38 - loss: 0.6415 - acc: 0.629 - ETA: 2:37 - loss: 0.6412 - acc: 0.629 - ETA: 2:37 - loss: 0.6410 - acc: 0.629 - ETA: 2:37 - loss: 0.6414 - acc: 0.629 - ETA: 2:36 - loss: 0.6410 - acc: 0.630 - ETA: 2:36 - loss: 0.6406 - acc: 0.630 - ETA: 2:35 - loss: 0.6402 - acc: 0.630 - ETA: 2:35 - loss: 0.6396 - acc: 0.631 - ETA: 2:35 - loss: 0.6393 - acc: 0.631 - ETA: 2:34 - loss: 0.6390 - acc: 0.631 - ETA: 2:34 - loss: 0.6387 - acc: 0.631 - ETA: 2:33 - loss: 0.6386 - acc: 0.631 - ETA: 2:33 - loss: 0.6388 - acc: 0.631 - ETA: 2:32 - loss: 0.6381 - acc: 0.631 - ETA: 2:32 - loss: 0.6377 - acc: 0.632 - ETA: 2:32 - loss: 0.6373 - acc: 0.632 - ETA: 2:31 - loss: 0.6377 - acc: 0.632 - ETA: 2:31 - loss: 0.6377 - acc: 0.632 - ETA: 2:31 - loss: 0.6370 - acc: 0.633 - ETA: 2:30 - loss: 0.6368 - acc: 0.633 - ETA: 2:30 - loss: 0.6371 - acc: 0.632 - ETA: 2:29 - loss: 0.6368 - acc: 0.633 - ETA: 2:29 - loss: 0.6366 - acc: 0.633 - ETA: 2:29 - loss: 0.6364 - acc: 0

19648/24394 [=======================>......] - ETA: 1:34 - loss: 0.6062 - acc: 0.664 - ETA: 1:34 - loss: 0.6061 - acc: 0.665 - ETA: 1:33 - loss: 0.6060 - acc: 0.665 - ETA: 1:33 - loss: 0.6057 - acc: 0.665 - ETA: 1:33 - loss: 0.6055 - acc: 0.665 - ETA: 1:33 - loss: 0.6054 - acc: 0.666 - ETA: 1:32 - loss: 0.6052 - acc: 0.666 - ETA: 1:32 - loss: 0.6048 - acc: 0.666 - ETA: 1:32 - loss: 0.6044 - acc: 0.666 - ETA: 1:32 - loss: 0.6043 - acc: 0.666 - ETA: 1:31 - loss: 0.6042 - acc: 0.666 - ETA: 1:31 - loss: 0.6039 - acc: 0.667 - ETA: 1:31 - loss: 0.6044 - acc: 0.666 - ETA: 1:30 - loss: 0.6046 - acc: 0.666 - ETA: 1:30 - loss: 0.6041 - acc: 0.667 - ETA: 1:30 - loss: 0.6041 - acc: 0.667 - ETA: 1:30 - loss: 0.6041 - acc: 0.667 - ETA: 1:29 - loss: 0.6039 - acc: 0.667 - ETA: 1:29 - loss: 0.6039 - acc: 0.667 - ETA: 1:29 - loss: 0.6036 - acc: 0.668 - ETA: 1:28 - loss: 0.6032 - acc: 0.668 - ETA: 1:28 - loss: 0.6029 - acc: 0.668 - ETA: 1:28 - loss: 0.6031 - acc: 0.668 - ETA: 1:28 - loss: 0.6030 - acc: 0

24394/24394 [==============================] - ETA: 38s - loss: 0.5884 - acc: 0.68 - ETA: 38s - loss: 0.5885 - acc: 0.68 - ETA: 38s - loss: 0.5885 - acc: 0.68 - ETA: 37s - loss: 0.5888 - acc: 0.68 - ETA: 37s - loss: 0.5887 - acc: 0.68 - ETA: 37s - loss: 0.5885 - acc: 0.68 - ETA: 36s - loss: 0.5885 - acc: 0.68 - ETA: 36s - loss: 0.5885 - acc: 0.68 - ETA: 36s - loss: 0.5887 - acc: 0.68 - ETA: 36s - loss: 0.5883 - acc: 0.68 - ETA: 35s - loss: 0.5884 - acc: 0.68 - ETA: 35s - loss: 0.5885 - acc: 0.68 - ETA: 35s - loss: 0.5885 - acc: 0.68 - ETA: 35s - loss: 0.5884 - acc: 0.68 - ETA: 34s - loss: 0.5883 - acc: 0.68 - ETA: 34s - loss: 0.5883 - acc: 0.68 - ETA: 34s - loss: 0.5883 - acc: 0.68 - ETA: 34s - loss: 0.5882 - acc: 0.68 - ETA: 33s - loss: 0.5880 - acc: 0.68 - ETA: 33s - loss: 0.5878 - acc: 0.68 - ETA: 33s - loss: 0.5877 - acc: 0.68 - ETA: 33s - loss: 0.5875 - acc: 0.68 - ETA: 32s - loss: 0.5874 - acc: 0.68 - ETA: 32s - loss: 0.5872 - acc: 0.68 - ETA: 32s - loss: 0.5873 - acc: 0.68 - ETA

...

@gaspsiagore &quot;stunningly beautiful&quot; and &quot;new piece&quot; don't usually go together. Wish I could go

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@mynameisgay You are so lucky you get to travel.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



TGIF. I don't feel good.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@reflective aww... sorry for both of you.  May the LORD give you strength and rest today!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Experiencing pain with paginating ASP.NET ListView controls

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@ShakeYourJunk

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



My chick wont shut up  Will only stop chirping if i sit with the bloody thing.

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



O dnt even like wearing  my skinny jean no more cuz of yall bitch ass 

@JinxCat What? I was being serious.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@semipenguin they are gone after I restarted TwetDeck  , and I wanted to follow  the peeps that I got posted frommy friends :/

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@CMiget That's always fun to clean up

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@ringqvist juice! I miss juice, any fruit

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



Broke Up with girlfriend, FEELING Very lonely and heartbroken  Sad times guys (N)

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



the mall i once worked at has turned into a very depressing place

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@daaku @danishk lol.. zzzzy in office and I'm alone in my bay again.

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



omg i was so nervous last block. i had my s

Actual emotion   :  negative



@carolkoh Sorry to hear that Carol.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



I've gotten a cold every time I've flown somewhere 6+ hrs away in the past year.  Hope to be back in the pool/on the bike next week.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@gonepie thats what doting husbands are for LOL!!  I hope that once my braces go on, I can eat normally again

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



just thought about how unfair 3 ppl have been 2 me n got depressed again  HATERS!!!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Have a slight headache

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@crapsticks how do i make it better after being robbed of 1m.

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@LiamJHayter  Chroma Key coolness ... When are the cumulus seminars going online couldn't see 

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



leaving now  off to get my angry teethed pulled....

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Then back to fucking school

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



Oh boy this is going to be a loooooong day

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@ThePaul it's a modded 360, so you can't send it back?  That Red Ring of Death sucks!  It took weeks to get mine back from Microsoft.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@Spiffums I wasn't near him.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



forgot my lunch and no one else is ordering stuff  boooooo!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@SheGotItSheBadd i've been up most of what normal ppl consider the night.  no problem going to sleep but nightmares been waking me up.

 # %s Predicted emotio

I was supported to do things I didn't like,, n when I was about to do what I intrested in,, mom always refused it...

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



English exam! Bleh i hate vocab

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@ToyStory2wasOk The party has to be moved to next weekend  ..stupid weather.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Just read McDonald's is actually running more ads than before the economic slowdown. Great. I'm lovin it.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



Really good concepts at #mozconcept. I *really* love to send something mine, but no time

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



he killed everyone for nothing. :o  :-ss

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
from collections import Counter
import sklearn
import xlrd
import nltk as nltk
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from os import path
from PIL import Image

In [ ]:
fig = plt.figure(figsize=(20,4))
sns.barplot(x = ProjectData['emotion'].unique(), y=ProjectData['emotion'].value_counts())
plt.show()